# Road Following 

Hope you have already performed basic motion, and teleoperation notebooks. So now, you would have fun working with JetBot motion, Camera usage and controlling JetBot with Joystick. 

If you have worked through "collision avoidance" notebook, your experience in collecting data will be very much helpful here. Also, we will be solving "road following" task using single camera with the help of AI deep learning tools. Through this excercise, you will see how to use neural network for regression tasks. We wil control JetBot's wheel angle to move smoothly around the track without loosing centerline.

In this notebook, we will explain data collection process for "road following" task, which can be used for Lego tracks, or to follow any lines or tracks in general. 

The way we will do this pretty simple!! We will manually place JetBot on the different lego pieces (straight, and turn slides). We will point JetBot in direction it should follow such that "X" (stored as steering value) position as how much JetBot move horizontally, and "Y" (stored as throttle value) position as how much farther JetBot can go following line of sight. Each Image is stored by the name: xy_SteeringVaue_ThrottleValue_UUID.jpeg 

That's all that we'll do in this notebook; data collection. Once we have good set of labelled images, we'll upload this data to a GPU enabled machine where we'll train a neural network to predict whether JetBot's steering angle control to follow center line of the road or track, based on the image it sees. We'll use this to implement a simple road following behavior in the end :)

Also, keep in mind that, we have three steps to complete this task:
1. Collect data with good variance in data ie. with different backgrounds (Follow: data_collection.ipynb (this notebook)) 
2. Train the neral network using PyTorch (Follow: train_model.ipynb)
3. Finally, test your deep learning model on JetBot (Follow: live_demo.ipynb)

### Import Libraries

So lets get started by importing all the required libraries for "data collection" purpose. We will mainly use OpenCV to visualize and save image with labels. Libraries such as uuid, datetime are used for image naming. 

In [ ]:
import traitlets
import os
from jetbot import Robot, Camera, bgr8_to_jpeg
import ipywidgets.widgets as widgets
from IPython.display import display
from uuid import uuid1
import json
import glob
import datetime
import numpy as np
import cv2
import time

### Display Live Camera Feed

First, let's initialize and display our camera like we did in the teleoperation notebook. 

We use Camera Class from JetBot to enable CSI MIPI camera. Our neural network takes a 224x224 pixel image as input. We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task). In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [ ]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='steering')
throttle_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='throttle')

def display_steering(camera_image):
    image = np.copy(camera_image)
    steering = steering_slider.value
    throttle = throttle_slider.value
    x = int(steering * 224 / 2 + 112)
    y = int(throttle * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_steering)

display(widgets.HBox([image_widget, target_widget]), steering_slider, throttle_slider)

### Create Gamepad Controller

This step is similar to "Teleoperation" task. In this task, we will use gamepad controller to label images.

The first thing we want to do is create an instance of the Controller widget, which we'll use to label images with "x" and "y" values as mentioned in introduction. The Controller widget takes a index parameter, which specifies the number of the controller. This is useful in case you have multiple controllers attached, or some gamepads appear as multiple controllers. To determine the index of the controller you're using,

Visit http://html5gamepad.com.
Press buttons on the gamepad you're using
Remember the index of the gamepad that is responding to the button presses
Next, we'll create and display our controller using that index.

In [ ]:
controller = widgets.Controller(index=0)

display(controller)

### Connect Gamepad Controller to Label Images

Now, even though we've connected our gamepad, we haven't yet attached the controller to label images! We'll connect that to the left and right vertical axes using the dlink function. The dlink function, unlike the link function, allows us to attach a transform between the source and target. 

In [ ]:
widgets.jsdlink((controller.axes[2], 'value'), (steering_slider, 'value'))
widgets.jsdlink((controller.axes[3], 'value'), (throttle_slider, 'value'))

Great Job! Now we will create folder and store labeled images. Images will be stored with the name " xy_SteeringVaue_ThrottleValue_UUID.jpeg". 

To make sure we don't repeat any file names (even across different machines!) we'll use the uuid package in python, which defines the uuid1 method to generate a unique identifier. This unique identifier is generated from information like the current time and the machine address.

Please keep following suggestions in mind while labelling data:
1. Keep one lego piece and collect data. Such that Neural Network is short sighted and do not rely on future lego pieces to take decision.
2. Try to change background as much as possible such that Nueral Network will learn features that matters for the task. 
3. Place Lego Pieces on different floors so that the background will change.
4. Have approximately equal number of images for JetBot in position where it needs to "Steer Right", "Steer Left" and "Move Forward". If you have more images for "steer right" as compared to "steer left" and "move forward" then Neural Network will learn to be biased for "steer right", which we do not want to happen.

For our experiments, we collected data while keeping one lego piece at a time on different background such as desk, kitchen floor, home carpet, and with different light variance such as under sunlight variations, yellow bulbs, and white light etc. We collected ~300 images where ~100 were each with "steer right" , "steer left", and "move forward". 

It's important to get varied data (as described by the above tips) and not just a lot of data. But don't worry, it goes pretty fast once you get going :)

WARNING: You need to check after training how many number of images are good for your use case. Above mentioned number of images worked well for our use case. You might need more or less based on effective data collection and labelling.



### Following images will show some of the examples.

#### Light Variations with Background changes

#### Steering Variations with Background changes

In [ ]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

for b in controller.buttons:
    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def steering_throttle_uuid(steering, throttle):
    steering_value = steering * 50 + 50
    throttle_value = throttle * 50 + 50
    return 'xy_%03d_%03d_%s' % (steering_value, throttle_value, uuid1())

def save_steering(change):
    if change['new']:
        uuid = steering_throttle_uuid(steering_slider.value, throttle_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[13].observe(save_steering, names='value')

display(widgets.VBox([
    target_widget,
    count_widget
]))

This is great! Now you can see in your browser that it created dataset folder. You can use the Jupyter Lab file browser to view these files!

REMINDER: You can move the widgets to new windows by right clicking the cell and clicking Create New View for Output. Or, you can just re-display them together as we will below

### Next

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training. First, we can call the following terminal command to compress our dataset folder into a single zip file.

The ! prefix indicates that we want to run the cell as a shell (or terminal) command.

The -r flag in the zip command below indicates recursive so that we include all nested files, the -q flag indicates quiet so that the zip command doesn't print any output

In [ ]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}

You should see a file named road_following_<Date&Time>.zip in the Jupyter Lab file browser. You should download the zip file using the Jupyter Lab file browser by right clicking and selecting Download.

Next, we'll need to upload this data to our GPU desktop or cloud machine (we refer to this as the host) to train the collision avoidance neural network. We'll assume that you've set up your training machine as described in the JetBot WiKi. If you have, you can navigate to http://<host_ip_address>:8888 to open up the Jupyter Lab environment running on the host. The notebook you'll need to open there is called collision_avoidance/train_model.ipynb.

So head on over to your training machine and follow the instructions there! Once your model is trained, we'll return to the robot Jupyter Lab enivornment to use the model for a live demo!